# Importing Libraries

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(1337)  # for reproducibility
import csv
import time


from keras.datasets import mnist


from keras.datasets import mnist
from keras.models import Sequential, model_from_json
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.utils import np_utils
from keras.regularizers import l2, activity_l2
from keras.layers.advanced_activations import LeakyReLU
from keras.utils.visualize_util import plot


In [ ]:
#os.chdir("/Users/Armin/Documents/Data_Science/Kaggle/Digit_Recognizer")
cwd=os.getcwd()
print "Working Dir Is:\n",cwd
print os.listdir(cwd+"/Data/")


In [ ]:
train_data = csv.reader(open(cwd+"/Data/train.csv","rb"),delimiter=',')
x = list(train_data)
train_XY= np.array(x[1:]).astype('float32')
test_data = csv.reader(open(cwd+"/Data/test.csv","rb"),delimiter=',')
x = list(test_data)
test_XY= np.array(x[1:]).astype('float32')

In [ ]:
batch_size = 128
nb_classes = 10
nb_epoch = 1

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = 32
nb_filters_2 = 64 
nb_filters_3 = 128 
# size of pooling area for max pooling
nb_pool = 2
# convolution kernel size
nb_conv = 3

In [ ]:
train_percentage = 0.7
train_size = int(np.round(train_XY.shape[0]*train_percentage))

In [ ]:
X_train = train_XY[0:train_size,1:]
print ("X_train size is:",X_train.shape)
y_train = train_XY[0:train_size,0]
print ("y_train size is:",y_train.shape)
X_test = train_XY[train_size:,1:   ]
print ("X_test size is:",X_test.shape)
y_test = train_XY[train_size:,0]
print ("y_test size is:",y_test.shape)

X_submission = test_XY
print ("X_submission size is:",X_submission.shape)

##### Adding the original MNIST data

In [ ]:
(X_train_MNIST, y_train_MNIST), (X_test_MNIST, y_test_MNIST) = mnist.load_data()

In [ ]:
# Reshape the 2D MNIST to 1D MNIST
X_train_MNIST = np.reshape(X_train_MNIST,(-1,784))
X_test_MNIST = np.reshape(X_test_MNIST,(-1,784))

In [ ]:
# Append it to Kaggle MNIST
X_train = np.concatenate((X_train,X_train_MNIST), axis=0)
y_train = np.concatenate((y_train,y_train_MNIST), axis=0)
X_test = np.concatenate((X_test,X_test_MNIST), axis=0)
y_test = np.concatenate((y_test,y_test_MNIST), axis=0)


print "Final X_train shape is: ",X_train.shape
print "Final X_test shape is: ",X_test.shape
print "Final y_train shape is: ",y_train.shape
print "Final y_test shape is: ",y_test.shape


In [ ]:
X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
X_submission = X_submission.reshape(X_submission.shape[0], 1, img_rows, img_cols)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_submission = X_submission.astype('float32')
X_train /= 255
X_test /= 255
X_submission /= 255

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

In [ ]:
y_train = np.asarray(y_train).astype('int8')
y_test = np.asarray(y_test).astype('int8')

In [ ]:
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [ ]:
model = Sequential()

model.add(Convolution2D(nb_filters, nb_conv, nb_conv,
                        border_mode='valid',
                        input_shape=(1, img_rows, img_cols)))
model.add(LeakyReLU(alpha=0.3))
model.add(Convolution2D(nb_filters, nb_conv, nb_conv))
model.add(LeakyReLU(alpha=0.3))
model.add(Convolution2D(nb_filters_3, nb_conv, nb_conv, activation="relu"))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(nb_filters_3, nb_conv, nb_conv, activation="relu"))
model.add(MaxPooling2D(strides=(2,2)))


#model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(LeakyReLU(alpha=0.3))
model.add(Dense(64))
model.add(LeakyReLU(alpha=0.3))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])


In [ ]:
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
print model.summary()

json_string = model.to_json()
open('my_model_architecture.json', 'w').write(json_string)
model.save_weights('my_model_weights.h5')

In [ ]:
# Reconstructing model in the future
#model = model_from_json(open('my_model_architecture.json').read())
#model.load_weights('my_model_weights.h5')
#model.compile(optimizer='adadelta', loss='categorical_crossentropy')


In [ ]:
prediction_results = model.predict_classes(X_submission, batch_size=32, verbose=1)

In [ ]:
prediction_results = prediction_results.astype('uint8')
print prediction_results

In [ ]:
save_name = 'prediction_results___'+time.strftime("H_%H_%M_%S___D_%d_%m_%Y")+'.csv'


In [ ]:
np.savetxt(save_name, np.c_[range(1,len(prediction_results)+1),prediction_results], delimiter=',', header = 'ImageId,Label', comments = '', fmt='%d')